In [1]:
pip install dash dash_bootstrap_components ta

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 7.0 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=8da7e74ffbd795f956ca3cd7bc104d381ebfb84b4556ed29d6ffb25a895bbc31
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [2]:
import os
import dash
import random
import warnings
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.express as px
import matplotlib.pyplot as plt
from dash import Dash, html, dcc
import dash_core_components as dcc
from plotly import graph_objs as go
from dash.exceptions import PreventUpdate
from sklearn.linear_model import LinearRegression
from dash.dependencies import Input, Output, State
from datetime import datetime as dt
from dash import html
import dash_bootstrap_components as dbc
import ta
from plotly.subplots import make_subplots
import plotly.tools as tls
from dash.dash_table import DataTable, FormatTemplate, Format


warnings.filterwarnings("ignore")

<ipython-input-2-5abcf3354152>:11: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [3]:
def trading_strategy(btc, stop_loss_threshold, strategy, buysignal = 30, Rf=0.0):

    # Define the trading signals
    btc['signal'] = 0.0

    if strategy == 'mavg':
        btc['signal'] = np.where(btc['short_mavg'] > btc['long_mavg'], 1.0, 0.0)
    elif strategy == 'rsi':
        btc['signal'] = np.where(btc['rsi'] < buysignal, 1.0, 0.0)
    elif strategy == 'bb':
        btc['signal'] = np.where(btc['bollinger_lband'] < btc['Price'], 1.0, 0.0)
    elif strategy == 'Price':
        btc.iloc[1:,-1] = 1.0



    # Generate trading orders
    btc['positions'] = btc['signal'].diff()

    # Set the initial capital
    initial_capital = float(10000.0)

    # Create a DataFrame to hold the portfolio information
    portfolio = pd.DataFrame(index=btc.index)
    portfolio['holdings'] = 0.0
    portfolio['cash'] = initial_capital
    portfolio['total'] = initial_capital
    portfolio['stop_loss_price'] = np.nan

    # Trailing stop loss and portfolio value update
    for i in range(1, len(portfolio)):
        # Carry forward the number of holdings and cash
        portfolio.at[portfolio.index[i], 'holdings'] = portfolio.at[portfolio.index[i-1], 'holdings']
        portfolio.at[portfolio.index[i], 'cash'] = portfolio.at[portfolio.index[i-1], 'cash']

        # Update the stop loss price if there is a position
        if portfolio.at[portfolio.index[i-1], 'holdings'] > 0:
            portfolio.at[portfolio.index[i], 'stop_loss_price'] = max(
                portfolio.at[portfolio.index[i-1], 'stop_loss_price'],
                btc['Price'].iloc[i] * stop_loss_threshold
            )

        # Check for stop loss trigger
        if portfolio.at[portfolio.index[i], 'holdings'] > 0 and btc['Price'].iloc[i] < portfolio.at[portfolio.index[i], 'stop_loss_price']:
            portfolio.at[portfolio.index[i], 'cash'] += portfolio.at[portfolio.index[i], 'holdings'] * btc['Price'].iloc[i]
            portfolio.at[portfolio.index[i], 'holdings'] = 0
            portfolio.at[portfolio.index[i], 'stop_loss_price'] = np.nan

        # Check for buy signal
        elif btc['positions'].iloc[i] == 1:
            portfolio.at[portfolio.index[i], 'holdings'] = portfolio.at[portfolio.index[i], 'cash'] / btc['Price'].iloc[i]
            portfolio.at[portfolio.index[i], 'cash'] = 0
            portfolio.at[portfolio.index[i], 'stop_loss_price'] = btc['Price'].iloc[i] * stop_loss_threshold

        # Check for sell signal
        elif btc['positions'].iloc[i] == -1 and portfolio.at[portfolio.index[i], 'holdings'] > 0:
            portfolio.at[portfolio.index[i], 'cash'] += portfolio.at[portfolio.index[i], 'holdings'] * btc['Price'].iloc[i]
            portfolio.at[portfolio.index[i], 'holdings'] = 0
            portfolio.at[portfolio.index[i], 'stop_loss_price'] = np.nan

        # Update total portfolio value
        portfolio.at[portfolio.index[i], 'total'] = portfolio.at[portfolio.index[i], 'cash'] + portfolio.at[portfolio.index[i], 'holdings'] * btc['Price'].iloc[i]


    # Calculate returns
    portfolio['returns'] = portfolio['total'].pct_change()

    # Calculate total and annualized return
    total_return_percentage = ((portfolio['total'].iloc[-1] - initial_capital) / initial_capital) * 100
    days_in_backtest = (portfolio.index[-1] - portfolio.index[0]).days
    annualized_return = ((portfolio['total'].iloc[-1] / initial_capital) ** (365.0 / days_in_backtest)) - 1

    # Calculate Buy & Hold return
    buy_hold_return = ((btc['Price'].iloc[-1] - btc['Price'].iloc[0]) / btc['Price'].iloc[0]) * 100

    # Calculate the number of round-trip trades
    round_trips = btc['positions'].abs().sum() / 2

    # Calculate maximum drawdown
    peak = portfolio['total'].cummax()
    drawdown = (portfolio['total'] - peak) / peak
    max_drawdown = drawdown.min()

    # Calculate winning trade percentage
    # Initialize a list to keep track of each round-trip trade's profit or loss
    trade_results = []

    # Track the entry (buy) and exit (sell) prices for each trade
    entry_price = 0.0
    exit_price = 0.0

    # Loop through the DataFrame to evaluate each trade
    for i in range(1, len(btc)):
        # Check for a buy signal
        if btc['positions'].iloc[i] == 1:
            entry_price = btc['Price'].iloc[i]

        # Check for a sell signal
        elif btc['positions'].iloc[i] == -1 and entry_price != 0:
            exit_price = btc['Price'].iloc[i]
            # Calculate the result of the trade (profit if positive, loss if negative)
            trade_results.append(exit_price - entry_price)
            # Reset entry price for the next trade
            entry_price = 0

    # Calculate the percentage of winning trades
    winning_trades = len([result for result in trade_results if result > 0])
    winning_trades_percentage = (winning_trades / len(trade_results)) * 100 if trade_results else 0

    # Set the annual risk-free rate
    annual_risk_free_rate = Rf

    # Calculate daily risk-free rate assuming 252 trading days in a year
    daily_risk_free_rate = (1 + annual_risk_free_rate)**(1/252) - 1

    # Calculate Sharpe Ratio for the strategy
    strategy_returns = portfolio['returns'].dropna()
    excess_daily_returns = strategy_returns - daily_risk_free_rate
    strategy_sharpe_ratio = excess_daily_returns.mean() / excess_daily_returns.std() * np.sqrt(252)

    # Calculate Buy & Hold Strategy Returns
    buy_hold_returns = btc['Price'].pct_change().dropna()
    excess_bh_daily_returns = buy_hold_returns - daily_risk_free_rate
    buy_hold_sharpe_ratio = excess_bh_daily_returns.mean() / excess_bh_daily_returns.std() * np.sqrt(252)

    # Calculate Cummulative return
    portfolio['returns'] = portfolio['returns'].fillna(0)
    portfolio['cumulative_return'] = (1 + portfolio['returns']).cumprod() - 1

    results_dict = {
        "Number of round-trip trades": int(round_trips),
        "Total Return": f"{total_return_percentage:.2f}",
        "Annualized Return": f"{annualized_return*100:.2f}",
        "Buy & Hold Return": f"{buy_hold_return:.2f}",
        "Winning Trades": f"{winning_trades_percentage:.2f}",
        "Maximum Drawdown": f"{max_drawdown*100:.2f}",
        "Strategy Sharpe Ratio": f"{strategy_sharpe_ratio:.2f}",
        "Buy & Hold Sharpe Ratio": f"{buy_hold_sharpe_ratio:.2f}",
        "Portfolio": portfolio
    }

    return results_dict

In [4]:
def generate_btc_prices_with_random_start_n_end(df, random_seed=1):
    btc_data = df.copy(deep=True)
    random.seed(random_seed)
    # Specify x and y for rows to remove randomly from top and bottom
    a = random.randint(0, len(btc_data)//2 - 1)
    b = random.randint(0, len(btc_data)//2 - 1)

    final_btc_data = df.iloc[a: -b if b != 0 else None]

    return final_btc_data

In [5]:
def bootstrap_resampling(prices, strategy, stop_loss_threshold=0.8, block_size=200, iterations=100, Rf=0.00, plot_result=False):
    df = prices.copy()
    bootstrap_results = []
    n_blocks = int(np.ceil(len(df) / block_size))  # Number of blocks needed to approximate the original data length

    # Prepare a DataFrame to store aggregated results, excluding the 'Portfolio' DataFrame
    aggregated_results = pd.DataFrame()

    for i in range(iterations):
        # Randomly choose starting indices for block sampling
        np.random.seed(i)
        block_start_indices = np.random.choice(len(df) - block_size + 1, n_blocks, replace=True)

        # Extract blocks and concatenate them
        blocks = [df.iloc[idx:idx + block_size] for idx in block_start_indices]
        resampled_data = pd.concat(blocks).reset_index(drop=True)

        # Ensure the resampled data is not longer than the original data
        resampled_data = resampled_data.iloc[:len(df)]

        # Fetch prices with random start and end
        price_copy2 = df.copy(deep=True)
        random_startend_prices = generate_btc_prices_with_random_start_n_end(price_copy2, i)

        # Apply the trading strategy to the resampled data
        resampled_data.index = df.index
        result = trading_strategy(random_startend_prices, stop_loss_threshold, strategy, Rf=Rf)
        bootstrap_results.append(result)

    if plot_result:
        plt.figure(figsize=(16, 10))

        # Plot each bootstrap sample's portfolio value
        for i, result in enumerate(bootstrap_results):
            portfolio_value = result['Portfolio']['total']
            plt.plot(portfolio_value, alpha=0.5)  # Alpha for transparency

        # Set the y-axis to be log-scaled
        plt.yscale('log')

        # Center y-axis at 10000 with a horizontal line
        plt.axhline(y=10000, color='gray', linestyle='--', lw=0.5)

        plt.title('Portfolio Value for Bootstrap Samples')
        plt.ylabel('Portfolio Value (log scaled)')

        # Remove the x-axis
        plt.gca().axes.xaxis.set_visible(False)

        plt.show()


    return bootstrap_results

In [6]:
def get_stock_data(ticker_input, start_date, end_date):
    # File name where the data will be stored
    filename = f"{ticker_input}.csv"

    print("Downloading data...")
    data = yf.download(ticker_input, start=start_date, end=end_date, progress=False)
    if not data.empty:
      data.to_csv(filename)
      print(f"Data downloaded and saved as {filename}")
    else:
      print("Failed to download data.")

    # # Check if the data already exists
    # if os.path.exists(filename):
    #     print("Loading data from existing file.")
    #     data = pd.read_csv(filename, index_col='Date', parse_dates=True)
    # else:
    #     print("Downloading data...")
    #     data = yf.download(ticker_input, start='1900-01-01', end='2030-12-31', progress=False)
    #     if not data.empty:
    #         data.to_csv(filename)
    #         print(f"Data downloaded and saved as {filename}")
    #     else:
    #         print("Failed to download data.")

    if not data.empty:
        data.index = data.index.tz_localize(None)
        # Subset the data to the specified date range before returning
        data = data[(data.index >= pd.to_datetime(start_date)) & (data.index <= pd.to_datetime(end_date))]


    data = data['Adj Close']

    data.rename(columns={ticker_input: 'Adj Close'}, inplace=True)

    return data

# ticker = "MSFT"
# start_date = '2021-01-01'  # Start date for the data
# end_date = '2022-12-31'  # End date for the data
# data = get_stock_data(ticker, start_date, end_date)
# print(data.head())

In [7]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div(
    [
        html.Div(
            [
                html.Center(html.H2('Trading Strategy'), className='theblues', style={'font-size': '25px'}),
                html.Div(
                    'Created by Aditya Bakshi',
                    style={
                        'textAlign': 'center',
                        'color': '#888888',
                        'fontSize': '14px',
                        'marginBottom': '20px'
                    }
                )
            ]
        ),
        html.Br(),
        html.Div(
            [
                html.Center(html.H4('Test Strategy'), className='theblues', style={'font-size': '20px'}),
                html.Div(
                    "Our app enables users to simulate three popular trading strategies: Moving Average, Bollinger Bands, and Relative Strength Index (RSI), each integrated with stop-loss mechanisms to enhance trading safety. By adjusting the strategies' parameters, users can explore and assess their performance in different market conditions. The app also features a robustness test using bootstrapping techniques, ensuring that the strategies are both reliable and effective across a range of scenarios. This comprehensive tool is designed for traders looking to refine their techniques and boost their market performance.",

                    style={
                        'width': '100%',
                        'fontSize': '16px',
                        'text-align': 'center'
                    },
                ),

                dcc.ConfirmDialog(
                    id='error-dialog',
                    message='An error occurred: please check your inputs (positive integer window periods, non-negative loss threshold between 0 and 1, non-negative buy-signal level and std, non-empty tickers).'
                ),

                dcc.ConfirmDialog(
                    id='error-dialog2',
                    message='An error occurred: please check your ticker input: whether it is invalid or empty.'
                ),
            ],
            style={'backgroundColor': '#ddedea', 'padding': '20px', 'borderRadius': '15px'}
        ),
        html.Br(),
        html.Div(
            [
                dbc.Container([
                    dbc.Row([
                        dbc.Col([
                            dbc.Label('Enter Ticker:', className='mb-2'),
                            dbc.Input(id='ticker-input', type='text', value='BTC-USD', placeholder='Enter ticker (e.g., AAPL, GOOGL)'),
                        ], width=12)
                    ], className='mb-3'),

                    dbc.Row([
                        dbc.Col([
                            dbc.Label('Stop Loss Threshold:', className='mb-2'),
                            dbc.Input(id='stop-loss-threshold', type='number', value=0.8, min=0, max=1.0),
                        ], width=6),
                        dbc.Col([
                            dbc.Label('Start Date:', className='mb-2'),
                            dcc.DatePickerSingle(id='start-date', min_date_allowed=dt(2014, 1, 1), max_date_allowed=dt.now(), initial_visible_month=dt(2014, 1, 1), date=dt(2014, 1, 1)),
                        ], width=3),
                        dbc.Col([
                            dbc.Label('End Date:', className='mb-2'),
                            dcc.DatePickerSingle(id='end-date', min_date_allowed=dt(2014, 1, 1), max_date_allowed=dt.now(), initial_visible_month=dt(2023, 10, 31), date=dt(2023, 10, 31)),
                        ], width=3)
                    ], className='mb-3'),

                    dbc.Row([
                        dbc.Col([
                            dbc.Checklist(
                                options=[
                                    {'label': 'Price', 'value': 'Price'},
                                    {'label': 'Moving Average', 'value': 'mavg'},
                                    {'label': 'RSI', 'value': 'rsi'},
                                    {'label': 'Bollinger Band', 'value': 'bb'},
                                ],
                                value=['Price', 'mavg', 'rsi', 'bb'],
                                id='display-options',
                                inline=True,
                                switch=True,
                            )
                        ], width=12)
                    ], className='mb-3'),

                    dbc.Row([
                        dbc.Col([
                            dbc.Label('Long Window:', className='mb-2'),
                            dbc.Input(id='long-window', type='number', value=26, min=0),
                        ], width=6),
                        dbc.Col([
                            dbc.Label('Short Window:', className='mb-2'),
                            dbc.Input(id='short-window', type='number', value=12, min=0),
                        ], width=6)
                    ], className='mb-3'),

                    dbc.Row([
                        dbc.Col([
                            dbc.Label('RSI window:', className='mb-2'),
                            dbc.Input(type='number', id='rsi-window', value=9, min=0),
                        ], width=6),
                        dbc.Col([
                            dbc.Label('RSI buy signal level:', className='mb-2'),
                            dbc.Input(type='number', id='rsi-buy-signal', value=30, min=0),
                        ], width=6)
                    ], className='mb-3'),

                    dbc.Row([
                        dbc.Col([
                            dbc.Label('Bollinger Band buy signal (# of std):', className='mb-2'),
                            dbc.Input(type='number', id='bb-window', value=1, min=0),
                        ], width=12)
                    ], className='mb-3'),

                    dbc.Button("Run Strategy", id='run-price-plot-btn', color="primary", className='mb-3'),

                ], fluid=True),

                dcc.Loading(
                    id="graph_ldg",
                    children=[html.Div(id='price-container', children=[dcc.Graph(id='price-graph')]),],
                    type="dot"),

                dcc.Loading(
                    id="graph_ldg_l",
                    children=[html.Div(id='values-container', style={'display': 'flex', 'flexDirection': 'column', 'margin': '10px'})],
                    type="dot",
                ),

                html.Br(),

                dcc.Loading(
                    id="graph_ldg_2",
                    children=[html.Div(id='plot-container', children=[dcc.Graph(id='strategy-plot')]),],
                    type="dot"),
            ],
            style={'backgroundColor': '#daeaf6', 'padding': '20px', 'borderRadius': '15px', 'boxShadow': '2px 2px 10px #AAA'}
        ),
        html.Br(),
        html.Hr(),
        html.Br(),
        html.Div(
            [
                html.Center(html.H4('Bootstrap strategy'), className='theblues', style={'font-size': '20px'}),
                html.Div(
                    "To verify the robustness of your trading strategy, our app allows you to utilize bootstrapping for a customizable number of simulations. Simply select your preferred strategy type (It will maintain the same parameters you've previously chosen). Then, specify the number of simulations you wish to run. This feature helps ensure that your strategy performs consistently well under various market conditions, providing you with confidence in its reliability and effectiveness.",
                    style={
                        'width': '100%',
                        'fontSize': '16px',
                        'text-align': 'center'
                    },
                )
            ],
            style={'backgroundColor': '#ddedea', 'padding': '20px', 'borderRadius': '15px'}
        ),
        html.Br(),
        html.Div(
            [
                dbc.Container([
                    dbc.Row([
                        dbc.Col([
                            dbc.Label("Select Strategy:", className="me-2"),
                            dbc.Select(
                                id='strategy-selection',
                                options=[
                                    {'label': 'Moving Averages (MAVG)', 'value': 'MAVG'},
                                    {'label': 'Bollinger Bands (BB)', 'value': 'BB'},
                                    {'label': 'Relative Strength Index (RSI)', 'value': 'RSI'}
                                ],
                                value='MAVG',
                                size="lg",
                            ),
                        ], width=6),
                        dbc.Col([
                            dbc.Label("Number of Simulations:", className="me-2"),
                            dbc.Input(
                                id='no-of-simulations',
                                type='number',
                                value=6,
                                min=5,
                                size="lg",
                            ),
                        ], width=3),
                    ], align='center', className="mb-3"),

                    dbc.Row([
                        dbc.Col([
                            dbc.Button(
                                "Simulate Strategy",
                                id='generate-plot-btn',
                                color="primary",
                                className="mb-3"
                            ),
                        ], width=4)
                    ], justify="start")
                ], fluid=True),
                dcc.Loading(
                    id="loading-1",
                    type="cube",
                    children=None
                )
            ],
            style={'backgroundColor': '#daeaf6', 'padding': '20px', 'borderRadius': '15px', 'boxShadow': '2px 2px 10px #AAA', 'height': 'auto'}
        ),
    ],
    style={
        'margin': '2em',
        'borderRadius': '1em',
        'padding': '1em',
        'backgroundColor': '#f0efeb'
    }
)


@app.callback(
    Output("loading-1", "children"),
    Input("generate-plot-btn", "n_clicks"),
    State('strategy-selection', 'value'),
    State('ticker-input', 'value'),
    State('long-window', 'value'),
    State('short-window', 'value'),
    State('rsi-window', 'value'),
    State('rsi-buy-signal', 'value'),
    State('bb-window', 'value'),

    State('stop-loss-threshold', 'value'),
    State('no-of-simulations','value'),
    prevent_initial_call=True
)
def update_graph(n_clicks, strategy, ticker_input, long_window, short_window, rsi_window, rsi_buy_signal, bb_window, stop_loss_threshold, no_of_simulations):
    if not n_clicks:
        return None

    try:
        #no_of_simulations = int(no_of_simulations)
        if no_of_simulations is None or no_of_simulations > 100:
            return html.Div([
                html.P("Error: Number of simulations must be between 6 and 100."),
                html.P("Please adjust the Number of Simulations value and try again.")
            ], style={'color': 'red', 'fontSize': '16px'})
    except ValueError:
        return html.Div([
            html.P("Error: Number of simulations must be a valid integer."),
            html.P("Please enter a valid integer for Number of Simulations.")
        ], style={'color': 'red', 'fontSize': '16px'})

    try:
        # Fetch and process data
        data = yf.download(ticker_input, start='2014-01-01', end='2024-10-31', progress=False)
        if data.empty:
            return html.Div("Error: No data returned from the API.", style={'color': 'red', 'fontSize': '16px'})

        data = data['Adj Close'].reset_index()
        data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
        data.set_index('Date', inplace=True)
        data.rename(columns={ticker_input: "Price"}, inplace=True)

        # Calculate indicators
        data['short_mavg'] = data['Price'].rolling(window=short_window, min_periods=1).mean()
        data['long_mavg'] = data['Price'].rolling(window=long_window, min_periods=1).mean()
        data['rsi'] = ta.momentum.RSIIndicator(data['Price'], window=rsi_window, fillna=True).rsi()
        indicator_bb = ta.volatility.BollingerBands(close=data['Price'], window=20, window_dev=bb_window)
        data['bollinger_hband'] = indicator_bb.bollinger_hband()
        data['bollinger_lband'] = indicator_bb.bollinger_lband()

        if strategy == 'MAVG':
            selected_data = data[['Price', 'short_mavg', 'long_mavg']]
        elif strategy == 'BB':
            selected_data = data[['Price', 'bollinger_hband', 'bollinger_lband']]
        else:
            selected_data = data[['Price', 'rsi']]

        # Perform bootstrapping
        bootstrap_results = bootstrap_resampling(selected_data, strategy=strategy.lower(), stop_loss_threshold=stop_loss_threshold, block_size=200, iterations=no_of_simulations, Rf=0.00, plot_result=False)

        fig = go.Figure()

        # Plot each bootstrap sample's portfolio value
        for i, result in enumerate(bootstrap_results):
            portfolio_value = result['Portfolio']['total']
            fig.add_trace(go.Scatter(x=portfolio_value.index, y=portfolio_value.values, mode='lines', name=f'Sample {i}', opacity=0.5))

        fig.update_layout(
            title=f'Portfolio Value for Bootstrap Samples ({strategy})',
            yaxis=dict(
                type='log',
                title='Portfolio Value (log scaled)'
            ),
            xaxis=dict(visible=False),
            showlegend=False,
            height=700
        )

        fig.add_hline(y=10000, line_dash="dash", line_color="gray", line_width=0.5)

        return dcc.Graph(figure=fig)
    except Exception as e:
        return html.Div(f"An unexpected error occurred: {str(e)}", style={'color': 'red', 'fontSize': '16px'})



@app.callback(
    [Output('ticker-input', 'value'),
     Output('long-window', 'value'),
     Output('short-window', 'value'),

     Output('rsi-window', 'value'),
     Output('rsi-buy-signal', 'value'),
     Output('bb-window', 'value'),

     Output('stop-loss-threshold', 'value'),
     Output('error-dialog', 'displayed')],
    [Input('run-price-plot-btn', 'n_clicks'),
     Input('error-dialog', 'cancel_n_clicks'),
     Input('error-dialog', 'submit_n_clicks')],
    [State('ticker-input', 'value'),
     State('long-window', 'value'),
     State('short-window', 'value'),

     State('rsi-window', 'value'),
     State('rsi-buy-signal', 'value'),
     State('bb-window', 'value'),

     State('stop-loss-threshold', 'value'),
     State('error-dialog', 'displayed')],
    prevent_initial_call=True
)
def handle_error_and_reset_inputs(n_clicks, cancel_n_clicks, submit_n_clicks, ticker_input,
                                  long_window, short_window, rsi_window, rsi_buy_signal, bb_window, stop_loss_threshold, displayed):
    ctx = dash.callback_context
    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if triggered_id == 'run-price-plot-btn' and n_clicks is not None:
        if (not ticker_input or
            long_window is None or long_window <= 0 or
            short_window is None or short_window <= 0 or

            rsi_window is None or rsi_window <= 0 or
            rsi_buy_signal is None or rsi_buy_signal <= 0 or
            bb_window is None or bb_window <= 0 or stop_loss_threshold is None):

            ticker_input = ticker_input or 'BTC-USD'
            long_window = 26
            short_window = 12

            rsi_window = 9
            rsi_buy_signal = 40
            bb_window = 1

            stop_loss_threshold = stop_loss_threshold or 0.8
            show_error_dialog = True
        else:
            show_error_dialog = False
    elif triggered_id in ['error-dialog', 'submit_n_clicks', 'cancel_n_clicks'] and displayed:
        show_error_dialog = False
    else:
        raise PreventUpdate

    return ticker_input, long_window, short_window, rsi_window, rsi_buy_signal, bb_window, stop_loss_threshold, show_error_dialog

@app.callback(
    [Output('price-graph', 'figure'),
     Output('values-container', 'children'),
     Output('strategy-plot', 'figure'),
     Output('error-dialog2', 'displayed'),
     Output('error-dialog2', 'message'),
    Output('ticker-input', 'value',allow_duplicate=True)],
    [Input('run-price-plot-btn', 'n_clicks')],
    [State('ticker-input', 'value'),
     State('long-window', 'value'),
     State('short-window', 'value'),
     State('rsi-window', 'value'),
     State('rsi-buy-signal', 'value'),
     State('bb-window', 'value'),

     State('stop-loss-threshold', 'value'),
     State('start-date', 'date'),
     State('end-date', 'date'),
     State('display-options', 'value')],
    prevent_initial_call=True,
)
def update_data_and_plots(n_clicks, ticker_input, long_window,
                          short_window, rsi_window, rsi_buy_signal, bb_window, stop_loss_threshold, start_date, end_date, display_options):
    if n_clicks is None:
        raise PreventUpdate

    if (not ticker_input or
            long_window is None or long_window <= 0 or
            short_window is None or short_window <= 0 or

            rsi_window is None or rsi_window <= 0 or
            rsi_buy_signal is None or rsi_buy_signal <= 0 or
            bb_window is None or bb_window <= 0 or


            stop_loss_threshold is None):
        raise PreventUpdate

    try:
        start_date = pd.to_datetime(start_date).strftime('%Y-%m-%d')
        end_date = pd.to_datetime(end_date).strftime('%Y-%m-%d')

        data = get_stock_data(ticker_input, start_date, end_date)

        data = data.reset_index()
        data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
        data.columns.name = None
        data.reset_index(drop=True, inplace=True)
        data.set_index('Date', inplace=True)
        data = data.rename(columns={"Adj Close": "Price"})

        # Recalculate the moving averages and stop-loss threshold
        data['short_mavg'] = data['Price'].rolling(window=short_window, min_periods=1).mean()
        data['long_mavg'] = data['Price'].rolling(window=long_window, min_periods=1).mean()

        data['rsi'] = ta.momentum.RSIIndicator(data['Price'], window=rsi_window, fillna=True).rsi()

        indicator_bb = ta.volatility.BollingerBands(close=data['Price'], window=20, window_dev=bb_window)
        data['bollinger_hband'] = indicator_bb.bollinger_hband()
        data['bollinger_lband'] = indicator_bb.bollinger_lband()

        asset_data_mavg = data[['Price','short_mavg','long_mavg']]
        asset_data_rsi = data[['Price','rsi']]
        asset_data_bb = data[['Price','bollinger_hband','bollinger_lband']]


        all_result_dict = {}
        for op in display_options:
            if op == 'Price':
                continue
            elif op == 'mavg':
                all_result_dict[op] = trading_strategy(asset_data_mavg, stop_loss_threshold, op, Rf = 0.00)
            elif op == 'rsi':
                all_result_dict[op] = trading_strategy(asset_data_rsi, stop_loss_threshold, op, buysignal = rsi_buy_signal, Rf = 0.00)
            elif op == 'bb':
                all_result_dict[op] = trading_strategy(asset_data_bb, stop_loss_threshold, op, Rf = 0.00)


        # combine the result of every selected strategy into "portfolio"

        portfolio = data[['Price']]
        for op, result_dict in all_result_dict.items():
            if op != 'Price':
                new_portfolio = result_dict['Portfolio'][['total']].rename(columns={'total': op})
                portfolio = pd.concat([portfolio, new_portfolio], axis=1, ignore_index=False)


        options = []
        for op in display_options:
            if op == 'Price':
                options.append('Price')
            elif op == 'mavg':
                options.extend(['short_mavg','long_mavg'])
            elif op =='rsi':
                options.append('rsi')
            elif op == 'bb':
                options.extend(['bollinger_hband','bollinger_lband'])

        # Update the figure

        fig_price = px.line(data, y=options)

        fig_price.update_layout(
            title={
                'text': f"Time Series Plot of {ticker_input}",
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            title_font=dict(size=20, color='black'),
            plot_bgcolor='white',
            xaxis=dict(
                showline=True,
                showgrid=False,
                title='Date',
                title_font={'size': 16},
                rangeselector=dict(
                    buttons=list([
                        dict(count=1, label="1m", step="month", stepmode="backward"),
                        dict(count=6, label="6m", step="month", stepmode="backward"),
                        dict(count=1, label="YTD", step="year", stepmode="todate"),
                        dict(count=1, label="1y", step="year", stepmode="backward"),
                        dict(step="all")
                    ]),
                    bgcolor='lightblue',
                    font=dict(color='darkblue', size=14)
                ),
                rangeslider=dict(
                    visible=False,
                    thickness=0.1,
                    bgcolor='lightgrey'
                ),
                type='date'
            ),
            yaxis=dict(
                title='Price ($)',
                title_font=dict(size=16, color='darkblue'),
                showgrid=True,
                showline=True,
            ),
            height=600
        )

        fig_price.update_xaxes(
            tickangle=45,
            tickmode='auto',
            nticks=20,
            linecolor='gray',
            linewidth=2
        )

        fig_price.update_yaxes(
            tickprefix="$",
            showline=True,
            linewidth=2,
            linecolor='gray'
        )

        rows = []
        for op in display_options:
            if op != 'Price':
                row = {
                    'Strategy': op,
                    'Round Trip Trades': all_result_dict[op]['Number of round-trip trades'],
                    'Total Return (%)': f"{all_result_dict[op]['Total Return']}%",
                    'Annualized Return (%)': f"{all_result_dict[op]['Annualized Return']}%",
                    'Winning Trades (%)': f"{all_result_dict[op]['Winning Trades']}%",
                    'Strategy Sharpe Ratio': all_result_dict[op]['Strategy Sharpe Ratio'],
                    'Maximum Drawdown (%)': f"{all_result_dict[op]['Maximum Drawdown']}%"
                }
                rows.append(row)
        results = pd.DataFrame(rows)
        export = True
        results_table = DataTable(
            id='table',
            data=results.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in results.columns],

            style_table={
                'overflowX': 'auto',
                'height': 'auto',
                'border': '1px solid lightgrey'
            },

            style_cell={
                'fontSize': '14px',
                'fontFamily': 'sans-serif',
                'textAlign': 'center',
                'height': 'auto',
                'minWidth': '100px', 'width': '150px', 'maxWidth': '200px',
                'whiteSpace': 'normal',
                'padding': '10px'
            },

            style_header={
                'backgroundColor': 'lightgrey',
                'fontWeight': 'bold',
                'border': '1px solid black',
                'fontSize': '16px',
                'padding': '10px'
            },

            style_data={
                'border': '1px solid grey'
            },

            sort_action='native',    # Enable sorting

            export_format='xlsx' if True else 'none',
            export_headers='display',
            css=[{
                'selector': '.export',
                'rule': 'color: white; background-color: #007BFF; border: none; padding: 8px 16px; margin: 8px; border-radius: 5px; cursor: pointer;'
            }],

            filter_options={'case': 'insensitive'}
        )


        # Update Strategy Plot

        figure_strategy = go.Figure()

        # Loop through each option in display_options
        for op in display_options:
            if op == 'Price':
                continue
            else:
                figure_strategy.add_trace(go.Scatter(x=portfolio.index,y=portfolio[op], mode='lines', name=f'Portfolio Value: {op}'))

        figure_strategy.add_trace(go.Scatter(
            x=data.index,
            y=data['Price'],
            mode='lines',
            name=f'{ticker_input} Price',
            line=dict(width=2, color='blue'),
            opacity=0.8
        ))

        figure_strategy.update_layout(
            title={
                'text': f"Strategy Performance of {ticker_input}",
                'y': 0.9,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            plot_bgcolor='white',
            xaxis=dict(
                showline=True,
                showgrid=False,
                title='Date',
                title_font={'size': 16},
                tickfont=dict(size=14, color='black'),
                rangeselector=dict(
                    buttons=list([
                        dict(count=1, label="1m", step="month", stepmode="backward"),
                        dict(count=6, label="6m", step="month", stepmode="backward"),
                        dict(count=1, label="YTD", step="year", stepmode="todate"),
                        dict(count=1, label="1y", step="year", stepmode="backward"),
                        dict(step="all")
                    ]),
                    bgcolor='lightblue',
                    font=dict(color='darkblue', size=14)
                ),
                rangeslider=dict(visible=False),
                type='date'
            ),
            yaxis=dict(
                title='Price ($)',
                title_font=dict(size=16, color='darkblue'),
                showgrid=True,
                gridcolor='lightgrey'
            ),
            height=600
        )

        figure_strategy.update_xaxes(
            tickangle=45,
            nticks=20,
            linecolor='gray',
            linewidth=2
        )

        figure_strategy.update_yaxes(
            tickprefix="$",
            showline=True,
            linewidth=2,
            linecolor='gray'
        )

        return fig_price, results_table, figure_strategy, False, '',ticker_input

    except Exception as e:
        if str(e) == "single positional indexer is out-of-bounds":
            error_message = 'The ticker you enter is invalid, please check your inputs.'
            ticker_input='BTC-USD'
        else:
            error_message = f"An error occurred: {str(e)}. Please check your inputs."
        return go.Figure(), [], go.Figure(), True, error_message,ticker_input


print('About to start...')
app.run_server(
    debug=True,
    port=8400
)

About to start...


<IPython.core.display.Javascript object>